In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [64]:
product_sales = pd.read_csv(r'C:\Users\Admin\RBAC\Round3_RBAC\ProductSales.csv')

In [65]:
product_map = pd.read_csv(r'C:\Users\Admin\RBAC\Round3_RBAC\ProductMapping.csv')
voucher_map = pd.read_csv(r'C:\Users\Admin\RBAC\Round3_RBAC\VoucherMapping.csv')

# 1 | Change product mapping

In [66]:
def split_prefix(x):
    return x.split('_')[-1]
product_map['NAME_NOPREFIX']  = product_map['NAME'].apply(split_prefix)
NAME_NONE_PREFIX = product_map[['FoodCode','NAME_NOPREFIX']]
# NAME_NONE_PREFIX.to_excel(r'Product_NonPrefix.xlsx')

# 2 | Bought together products

In [67]:
# food code to exclude
name_non_prefix = ['ADD ON', 'KEYNOTE', 'PROMOTION', 'REDEEM VOUCHER', 'UP SIZE', 'COMBO']
food_code_to_get = product_map[product_map['NAME_NOPREFIX'].isin(name_non_prefix) == 0]['FoodCode']
product_sales_new = product_sales[product_sales['FoodCode'].isin(food_code_to_get)]

In [68]:
NAME_NONE_PREFIX['NAME_NOPREFIX'].sort_values().unique()

array(['7UPBOTTLE', '7UPCAN', '7UPGLASS', 'ADD ON', 'AQUAFINA',
       'BACON BREAD', 'BACONBLACKPEPPER', 'BACONCARBONARA', 'BACONONION',
       'BAKEDCHEESYCORNWITHBACON', 'BAKEDPOTATO', 'BBQCHICKENWINGS',
       'BEEF', 'BEEFRICE', 'BEEFTERIYAKI', 'BOLOGNESE', 'CALAMARI',
       'CHEESELOVERS', 'CHEESYPOPS', 'CHEESYRINGS', 'CHICKENDELUXE',
       'CHICKENSALADWITHPESTOSAUCE', 'CHICKENWITHCHEESESTYLE',
       'CHICKENWITHFOURCHEESE', 'COKE LIGHT CAN', 'COMBO',
       'COOKIE YOGURT', 'CREAMYCHICKENSOUP', 'CREAMYSHRIMPANDSAUSAGE',
       'CRINKLECUTFRENCHFRIES', 'CRISPYSALMONSKINSALAD', 'DELUXEPLATTER',
       'ENGLISH HOT TEA', 'FIESTASEAFOOD', 'FISHERMANTUNA',
       'FOURCHEESECHICKENWING', 'FRESH MINT LEMON',
       'FRIEDCHICKENWITHKOREANSPICYSAUCE',
       'FRIEDCHICKENWITHSALTEDEGGSAUCE', 'GARLICBREAD', 'GRABSTICK',
       'HAWAIIAN', 'HEINEKKENCAN', 'HONEYMUSTARDFRIES', 'ICECREAM',
       'ITALIANSAUAGES', 'KEYNOTE', 'LEMONSODA', 'LIPTONBOTTLE',
       'LIPTONGLASS', 'MARINARAS

In [69]:
# beverage code to exclude
beverage_prefix = ['7UPBOTTLE', '7UPCAN', '7UPGLASS', 
                   'AQUAFINA', 'COKE LIGHT CAN', 'ENGLISH HOT TEA', 
                   'LEMONSODA', 'LIPTONBOTTLE', 'LIPTONGLASS', 
                   'MIRINDABOTTLE', 'MIRINDAGLASS', 'MIRINDAORANGECAN', 'MIRINDASODACAN', 'MIRINDASODAGLASS', 
                   'PINE MANGO CRUSH', 'PEPSIBOTTLE', 'PEPSICAN', 'PEPSIGLASSS', 'PEPSILEMONCAN','HEINEKKENCAN',
                   'FRESH MINT LEMON','PEPSIBLACKGLASS','PEPSIZEROCAN','TRAVAI','TRASUATRADEN','TRADAOHATHI']
food_code_to_get = product_map[product_map['NAME_NOPREFIX'].isin(beverage_prefix) == 0]['FoodCode']
product_sales_new = product_sales_new[product_sales_new['FoodCode'].isin(food_code_to_get)]

In [94]:
merged = product_sales_new.merge(NAME_NONE_PREFIX, on = 'FoodCode')

In [95]:
list_of_item_list = []
for order_id, remaining_df in merged.groupby('BillID'):
    list_of_item_list.append(list(remaining_df.drop_duplicates(subset='NAME_NOPREFIX')['NAME_NOPREFIX'].values))
list_of_item_list = [list_of_item for list_of_item in list_of_item_list if len(list_of_item) > 1]

In [116]:
from itertools import combinations

com_dict = {}
for list_of_item in list_of_item_list:
    for com in combinations(list_of_item,2):
        if com in com_dict: com_dict[com] +=1
        else: com_dict[com] = 1

res_df = pd.DataFrame(pd.Series(com_dict)).reset_index().rename(columns={'level_0':'NAME_NOPREFIX', "level_1": "NAME_2", 0: "Concurrence"}).sort_values('Concurrence',ascending=False)
top10 = res_df[:10]
top10

,NAME_NOPREFIX,NAME_2,Concurrence
140,CHEESELOVERS,SFPESTO,6636
13,SFPESTO,CHEESELOVERS,6324
2,CRINKLECUTFRENCHFRIES,SFPESTO,5352
1741,SFPESTO,SPICYBEEF,4379
152,CHEESELOVERS,MEATLOVERS,4046
25,SFPESTO,OCEANDELIGHT,3959
128,SFPESTO,PEPPERONI,3696
10,PEPPERONI,SFPESTO,3522
51,SFPESTO,HAWAIIAN,3321
81,CRINKLECUTFRENCHFRIES,CHEESELOVERS,3277


In [124]:
temp = product_map.merge(res_df, on ='NAME_NOPREFIX')
temp['max_concur'] =  temp.groupby('NAME_NOPREFIX',as_index=False)['Concurrence'].transform('max')
temp[temp['Concurrence'] == temp['max_concur']].drop_duplicates(subset=['NAME_NOPREFIX'])[['FoodCode','NAME_NOPREFIX','NAME_2','Concurrence']]

,FoodCode,NAME_NOPREFIX,NAME_2,Concurrence
0,2294,VEGGARLIC,SFPESTO,385
5350,472,FRIEDCHICKENWITHKOREANSPICYSAUCE,SPICYBEEF,60
5548,1041,CHEESELOVERS,SFPESTO,6636
16040,1541,PEPPERONI,SFPESTO,3522
24566,886,HAWAIIAN,SFPESTO,2510
...,...,...,...,...
145816,785,FRIEDCHICKENWITHSALTEDEGGSAUCE,SFPESTO,62
146008,2809,BEEFRICE,SPICYPRAWN,3
146026,2344,PUMPKINSOUP,FISHERMANTUNA,2
146044,2198,CREAMYSHRIMPANDSAUSAGE,CHEESELOVERS,14
